# Model Training: Ridge, Random Forest, and LightGBM

This notebook documents the machine learning model training workflow for free testosterone estimation.

**Models Trained:**
1. **Ridge Regression** - Linear baseline with L2 regularization
2. **Random Forest** - Ensemble of decision trees for nonlinear patterns
3. **LightGBM** - Gradient boosting for best performance

**Approach:** Hybrid mechanistic + ML using Vermeulen FT as a baseline feature.

In [ ]:
import sys
from pathlib import Path

# Add project root to path for imports
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from freeT.train import (
    create_features,
    stratified_split,
    train_ridge,
    train_random_forest,
    train_lightgbm,
    cross_validate_model,
    save_model
)

print("Training modules imported successfully!")

## 1. Load and Prepare Data

Load the cleaned NHANES dataset. If real data is not available, we generate synthetic data for demonstration.

In [ ]:
# Try to load real NHANES data, fall back to synthetic if unavailable
data_path = project_root / 'data' / 'processed' / 'nhanes_combined.csv'

if data_path.exists():
    print(f"Loading real NHANES data from {data_path}")
    df = pd.read_csv(data_path)
    print(f"Loaded {len(df)} records")
else:
    print("NHANES data not found. Generating synthetic data for demonstration...")
    np.random.seed(42)
    n_samples = 1000
    
    # Generate physiologically realistic data
    df = pd.DataFrame({
        'seqn': range(1, n_samples + 1),
        'tt_nmoll': np.random.uniform(5, 30, n_samples),      # TT: 5-30 nmol/L
        'shbg_nmoll': np.random.uniform(10, 80, n_samples),   # SHBG: 10-80 nmol/L
        'alb_gl': np.random.uniform(38, 50, n_samples)        # Albumin: 38-50 g/L
    })
    print(f"Generated {len(df)} synthetic records")

print(f"\nData shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
df.head()

## 2. Feature Engineering

Create features including:
- Raw biomarker values (TT, SHBG, Albumin)
- Derived features (SHBG/TT ratio)
- Hybrid feature: Vermeulen FT estimate as baseline

In [ ]:
# Create feature matrix
X, feature_names = create_features(df)

print(f"Feature matrix shape: {X.shape}")
print(f"\nFeatures ({len(feature_names)}):")
for i, name in enumerate(feature_names):
    print(f"  {i+1}. {name}: mean={X[:, i].mean():.4f}, std={X[:, i].std():.4f}")

## 3. Train/Test Split with Stratification

Split data ensuring balanced representation of SHBG tertiles (low, medium, high).

In [ ]:
# Stratified split by SHBG tertiles
X_train, X_test, y_train, y_test = stratified_split(df, test_size=0.3, random_state=42)

print(f"Training set: {X_train.shape[0]} samples ({X_train.shape[0]/len(df)*100:.1f}%)")
print(f"Test set:     {X_test.shape[0]} samples ({X_test.shape[0]/len(df)*100:.1f}%)")
print(f"Features:     {X_train.shape[1]}")

# Create validation set from training for early stopping
val_size = int(0.15 * len(X_train))
X_val, y_val = X_train[:val_size], y_train[:val_size]
X_train_full, y_train_full = X_train, y_train
X_train_lgb, y_train_lgb = X_train[val_size:], y_train[val_size:]

print(f"\nFor LightGBM early stopping:")
print(f"  Training: {X_train_lgb.shape[0]} samples")
print(f"  Validation: {X_val.shape[0]} samples")

## 4. Train Models

### 4.1 Ridge Regression Baseline

In [ ]:
# Train Ridge regression
print("Training Ridge Regression (alpha=1.0)...")
ridge_model = train_ridge(X_train_full, y_train_full, alpha=1.0)

# Evaluate on test set
ridge_pred = ridge_model.predict(X_test)
ridge_rmse = np.sqrt(np.mean((y_test - ridge_pred)**2))
ridge_mae = np.mean(np.abs(y_test - ridge_pred))

print(f"\nRidge Test Performance:")
print(f"  RMSE: {ridge_rmse:.6f} nmol/L")
print(f"  MAE:  {ridge_mae:.6f} nmol/L")

### 4.2 Random Forest

In [ ]:
# Train Random Forest
print("Training Random Forest (n_estimators=200)...")
rf_model = train_random_forest(X_train_full, y_train_full, n_estimators=200)

# Evaluate on test set
rf_pred = rf_model.predict(X_test)
rf_rmse = np.sqrt(np.mean((y_test - rf_pred)**2))
rf_mae = np.mean(np.abs(y_test - rf_pred))

print(f"\nRandom Forest Test Performance:")
print(f"  RMSE: {rf_rmse:.6f} nmol/L")
print(f"  MAE:  {rf_mae:.6f} nmol/L")

# Feature importance
print(f"\nFeature Importances:")
for name, importance in sorted(zip(feature_names, rf_model.feature_importances_), 
                               key=lambda x: x[1], reverse=True):
    print(f"  {name}: {importance:.4f}")

### 4.3 LightGBM with Early Stopping

In [ ]:
# Train LightGBM with early stopping
print("Training LightGBM with early stopping (patience=20)...")
lgb_model = train_lightgbm(X_train_lgb, y_train_lgb, X_val, y_val)

# Evaluate on test set
lgb_pred = lgb_model.predict(X_test)
lgb_rmse = np.sqrt(np.mean((y_test - lgb_pred)**2))
lgb_mae = np.mean(np.abs(y_test - lgb_pred))

print(f"\nLightGBM Test Performance:")
print(f"  RMSE: {lgb_rmse:.6f} nmol/L")
print(f"  MAE:  {lgb_mae:.6f} nmol/L")
print(f"  Best iteration: {lgb_model.best_iteration_}")

## 5. Cross-Validation Results

Evaluate models using 10-fold cross-validation for robust performance estimates.

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

# Use all features and targets for CV
X_all, _ = create_features(df)
y_all = X_all[:, feature_names.index('ft_vermeulen')]  # Use Vermeulen as target proxy

print("Running 10-fold cross-validation...\n")

# Ridge CV
print("Ridge Regression:")
ridge_cv = cross_validate_model(Ridge(alpha=1.0), X_all, y_all, n_splits=10)
print(f"  RMSE: {ridge_cv['RMSE_mean']:.6f} ± {ridge_cv['RMSE_std']:.6f}")
print(f"  MAE:  {ridge_cv['MAE_mean']:.6f} ± {ridge_cv['MAE_std']:.6f}")

# Random Forest CV
print("\nRandom Forest:")
rf_cv = cross_validate_model(RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1), 
                             X_all, y_all, n_splits=10)
print(f"  RMSE: {rf_cv['RMSE_mean']:.6f} ± {rf_cv['RMSE_std']:.6f}")
print(f"  MAE:  {rf_cv['MAE_mean']:.6f} ± {rf_cv['MAE_std']:.6f}")

## 6. Model Comparison Summary

In [ ]:
# Create comparison table
results = pd.DataFrame({
    'Model': ['Ridge', 'Random Forest', 'LightGBM'],
    'Test RMSE': [ridge_rmse, rf_rmse, lgb_rmse],
    'Test MAE': [ridge_mae, rf_mae, lgb_mae],
    'CV RMSE': [ridge_cv['RMSE_mean'], rf_cv['RMSE_mean'], np.nan],
    'CV RMSE std': [ridge_cv['RMSE_std'], rf_cv['RMSE_std'], np.nan]
})

print("="*70)
print("MODEL COMPARISON SUMMARY")
print("="*70)
print(results.to_string(index=False))
print("="*70)

# Identify best model
best_idx = results['Test RMSE'].idxmin()
best_model_name = results.loc[best_idx, 'Model']
print(f"\nBest model by Test RMSE: {best_model_name}")

In [ ]:
# Visualization: Predicted vs Actual
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

models = [('Ridge', ridge_pred), ('Random Forest', rf_pred), ('LightGBM', lgb_pred)]
colors = ['#2E86AB', '#A23B72', '#F18F01']

for ax, (name, pred), color in zip(axes, models, colors):
    ax.scatter(y_test, pred, alpha=0.5, s=15, c=color)
    lims = [min(y_test.min(), pred.min()), max(y_test.max(), pred.max())]
    ax.plot(lims, lims, 'k--', alpha=0.5, label='Identity')
    ax.set_xlabel('Actual FT (nmol/L)')
    ax.set_ylabel('Predicted FT (nmol/L)')
    ax.set_title(name)
    ax.set_aspect('equal')
    ax.legend()

fig.suptitle('Predicted vs Actual Free Testosterone', fontsize=12)
plt.tight_layout()
plt.show()

## 7. Save Best Model

In [ ]:
# Create models directory
models_dir = project_root / 'models'
models_dir.mkdir(exist_ok=True)

# Save all models
print("Saving models...")
save_model(ridge_model, str(models_dir / 'ridge_model.joblib'))
print(f"  Saved: {models_dir / 'ridge_model.joblib'}")

save_model(rf_model, str(models_dir / 'random_forest_model.joblib'))
print(f"  Saved: {models_dir / 'random_forest_model.joblib'}")

save_model(lgb_model, str(models_dir / 'lightgbm_model.joblib'))
print(f"  Saved: {models_dir / 'lightgbm_model.joblib'}")

# Save best model with explicit name
best_models = {'Ridge': ridge_model, 'Random Forest': rf_model, 'LightGBM': lgb_model}
best_model = best_models[best_model_name]
save_model(best_model, str(models_dir / 'best_model.joblib'))
print(f"\nBest model ({best_model_name}) saved as: {models_dir / 'best_model.joblib'}")

## 8. Summary

### Key Findings

1. **Hybrid Approach**: Using Vermeulen FT as a baseline feature provides a strong foundation
2. **Model Performance**: All models achieve low RMSE due to the informative Vermeulen feature
3. **Best Model**: Selected based on test set RMSE for deployment

### Next Steps

- Validate on external dataset (EMAS) using ED-measured FT as ground truth
- Compute Bland-Altman statistics and Lin's CCC for agreement analysis
- Perform subgroup analysis by SHBG tertiles

In [ ]:
print("Notebook execution complete.")